In [63]:
import tensorflow.keras
filepath = "C:/Users/Aaron Shi/dev/python/bluestamp/race_1"
loaded_model = tensorflow.keras.models.load_model(filepath)

In [4]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np

from pathlib import Path

#from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import glob
import scipy.io
import cv2

import random
import sys
import warnings 
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline



In [5]:
input_path = "C:/Users/Aaron Shi/dev/python/bluestamp/utk_data_2.mat"
temp = scipy.io.loadmat(input_path)
#print(temp)
images = temp['images']
ages = temp['ages']
genders = temp['genders']
races = temp['races']
image_size = temp['img_size']


In [6]:
import random
#(StartAgeRange, EndAgeRange, Label)
#infant, toddler, grade-schooler, adolescent, young-adult, adult, middle-aged, aged, elderly
#age_ranges=[(0,1,0), (2,5,1),(6,12,2),(13,18,3),(19,26,4), (27,45,5), (46, 64,6), (65, 79,7), (80, 120,8)]

age_classes = ["infant", "toddler", "grade-schooler", "adolescent", "young-adult", "adult", "middle-aged", "aged", "elderly"]

In [44]:

X_data = images
y_data_g = to_categorical(genders, 2)
y_data_a = to_categorical(ages, 9)
y_data_r = to_categorical(races, 5)

In [45]:
X_data = np.array(X_data)
#X_data = X_data.reshape(X_data.shape[0], (64, 64,3))
X_data = X_data.astype("float32")
X_data /= 255

In [16]:
a = random.sample(range(int(len(X_data))),(int(len(X_data)*0.75)))
print(len(X_data))

23707


In [10]:
def split_data(data):
    data_train = []
    data_test = []
    for i, val in enumerate(data):
        if i in a:
            data_train.append(val)
        else:
            data_test.append(val)
    return data_train, data_test

X_train, X_test = split_data(X_data)

In [46]:
X_train, X_test = split_data(X_data)
g_train, g_test = split_data(y_data_g[0])
a_train, a_test = split_data(y_data_a[0])
r_train, r_test = split_data(y_data_r[0])

In [47]:
X_train = np.array(X_train)
a_train = np.array(a_train)
g_train = np.array(g_train)
r_train = np.array(r_train)
X_test = np.array(X_test)
a_test = np.array(a_test)
g_test = np.array(g_test)
r_test = np.array(r_test)

In [48]:
X_train = X_train.reshape((17780, 64, 64, 1))
#a_train = a_train.reshape((17780, 64, 64, 1))
#g_train = g_train.reshape((1, 64, 64, 1))
#r_train = r_train.reshape((11853, 64, 64, 1))
X_test = X_test.reshape((5927, 64, 64, 1))
#a_test = a_test.reshape((11854, 64, 64, 1))
#g_test = g_test.reshape((11854, 64, 64, 1))
#r_test = r_test.reshape((11854, 64, 64, 1))


In [49]:
X_data = np.array(X_data).reshape(23707, 64, 64, 1)
y_data_g =  np.array(y_data_g[0])
y_data_a =  np.array(y_data_a[0])
y_data_r =  np.array(y_data_r[0])


In [50]:
y_data_g

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [64]:
score = loaded_model.evaluate(X_data, y_data_r, verbose=0)
print ("model %s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

model acc: 87.59%


In [65]:
y_prob = loaded_model.predict(X_data, batch_size=32, verbose=0)
y_pred = [np.argmax(prob) for prob in y_prob]
y_true = [np.argmax(true) for true in y_data_r]

In [66]:
from sklearn import metrics

#y_pred = loaded_model.predict(X_test)
    
print ("Classification Report:")
print (metrics.classification_report(y_true, y_pred))
print ("Confusion Matrix:")
print (metrics.confusion_matrix(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.96      0.90     10077
           1       0.95      0.89      0.92      4528
           2       0.97      0.86      0.91      3434
           3       0.86      0.85      0.86      3976
           4       0.77      0.41      0.53      1692

    accuracy                           0.88     23707
   macro avg       0.88      0.79      0.82     23707
weighted avg       0.88      0.88      0.87     23707

Confusion Matrix:
[[9705   75   39  187   71]
 [ 344 4024   19   91   50]
 [ 358   25 2944   76   31]
 [ 421   78   20 3399   58]
 [ 725   49   28  196  694]]
